In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [ ]:
#Example 2

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)

group = df.groupby(('cyl', 'mfr'))
source = ColumnDataSource(group)

p = figure(plot_width=800, plot_height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('cyl_mfr', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df.cyl.unique()), end=1)

p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="black", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("MPG", "@mpg_mean"), ("Cyl, Mfr", "@cyl_mfr")]))

show(p)


In [4]:
from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version
print(ipython_version)
print(pandas_version)
print(bokeh_version)

6.2.1
0.22.0
0.12.13


In [11]:
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with a size, color, and alpha
p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, line_color="navy", fill_color="black", fill_alpha=0.5)

show(p) # show the results

In [6]:
# Here is a list of categorical values (or factors)
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']

# Set the x_range to the list of categories above
p = figure(x_range=fruits, plot_height=250, title="Fruit Counts")

# Categorical values can also be used as coordinates
p.vbar(x=fruits, top=[5, 3, 4, 2, 4, 6], width=0.9)

# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [2]:
import json, urllib.request
import numpy as np
import pandas as pd

#loading in the goalie data
GoalieList = "http://www.nhl.com/stats/rest/goalies?isAggregate=false&reportType=goalie_basic&isGame=false&reportName=goaliesummary&sort=[{%22property%22:%22wins%22,%22direction%22:%22DESC%22}]&cayenneExp=gameTypeId=%222%22%20and%20seasonId%3E=20172018%20and%20seasonId%3C=20172018"
with urllib.request.urlopen(GoalieList) as url:  # getting the info into a dictionary
    goalie_data = json.loads(url.read().decode())
goalie_names = []
#goalie_arrays = np.zeros((len(goalie_data['data']), 8))
goalie_arrays = np.zeros((8, 95))

for i in range(95): #only doing the top 40 goalies
#for i in range(len(goalie_data['data'])):
    goalie_name = goalie_data['data'][i]['playerName']
    #print(goalie_name)
    save_pctg = goalie_data['data'][i]['savePctg']
    saves = goalie_data['data'][i]['saves'] 
    shotsAgainst = goalie_data['data'][i]['shotsAgainst']
    goalsAgainst = goalie_data['data'][i]['goalsAgainst']
    goals_against_average = goalie_data['data'][i]['goalsAgainstAverage']
    gamesPlayed = goalie_data['data'][i]['gamesPlayed']
    wins = goalie_data['data'][i]['wins']
    losses = goalie_data['data'][i]['losses']
    
    goalie_names.append(goalie_name)
    goalie_arrays[0][i] = wins
    goalie_arrays[1][i] = losses
    goalie_arrays[2][i] = gamesPlayed
    goalie_arrays[3][i] = goals_against_average
    goalie_arrays[4][i] = goalsAgainst
    goalie_arrays[5][i] = shotsAgainst
    goalie_arrays[6][i] = save_pctg
    goalie_arrays[7][i] = saves
    
dict_GP = {'name':goalie_names, 'GP':goalie_arrays[2]}
df_GP = pd.DataFrame(data=dict_GP)
df_GP.sort_values(by=['GP'], ascending = True, inplace = True)
#df_GAA.describe()
#print(df_GAA)

In [17]:

p = figure(x_range = goalie_names, plot_height = 300, plot_width = 900, title = "Games Played")

#Sorted by GamesPlayed from (most to least)

source = ColumnDataSource(data=dict(name=df_GP['name'].values, 
                                    GP=df_GP['GP'].values, 
                                    save_pctg=goalie_arrays[6], 
                                   ))

p.vbar(x = 'name', top='GP', width=.9, source=source)

p.y_range.start = 0

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Goalie Names"
p.xaxis.major_label_orientation = 1.2

p.add_tools(HoverTool(tooltips=[("GP", "@GP"), ("Name", "@name"), ("Save %", "@save_pctg")]))

show(p)
